In [ ]:
import requests
import numpy as np


class Digits(CFTarget):
    name = "digits_blackbox"
    endpoint = "http://localhost/predict"
    data_type = "image"
    input_shape = (1, 28, 28)
    output_classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    samples = []

    def load(self):
        self.samples = np.load("mnist_784.npz", allow_pickle=True)["X"] # float in [0,255] 

    def predict(self, x):
        x = np.array(x).astype(np.uint8).astype(np.float)  # quantize to valid range
        scores = self.predict_proba(x.reshape(x.shape[0], -1))
        # return a list of class probabilities; each row must be the same length as target_output_classes
        return scores.tolist()


from transformers import AutoFeatureExtractor, DeiTForImageClassificationWithTeacher
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-224')
model = DeiTForImageClassificationWithTeacher.from_pretrained('facebook/deit-base-distilled-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")

# forward pass
outputs = model(**inputs)
logits = outputs.logits

# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [ ]:
attack = "Blur"
target = Digits()
target.load()

In [ ]:
Counterfit.get_frameworks().keys()

In [ ]:
cfattack = Counterfit.build_attack(target, attack)
results = Counterfit.run_attack(cfattack)

In [ ]:
# set attack and target
attack = "hop_skip_jump"
target = CreditFraud()
target.load()

In [ ]:
# Individual attack
cfattack = Counterfit.build_attack(target, attack)
results = Counterfit.run_attack(cfattack)

In [ ]:
# Optimized attack
from counterfit.core.utils import set_id
scan_id = set_id()
optuna_study = optimize.optimize(scan_id, target, attack, num_iters=5)
print(optuna_study.best_trials[0].params)

In [ ]:
# Attack with best params
cfattack = Counterfit.attack_builder(target, attack)
cfattack.options.update_attack_parameters(optuna_study.best_trials[0].params)
results = Counterfit.attack_runner(cfattack)
cfattack.logger.num_queries

In [ ]:
from optuna.visualization import plot_contour
plot_contour(optuna_study, target=lambda t: t.values[0])

In [ ]:
from optuna.visualization import plot_intermediate_values
plot_intermediate_values(optuna_study)

In [ ]:
from optuna.visualization import plot_parallel_coordinate
plot_parallel_coordinate(optuna_study, target=lambda t: t.values[1])

In [ ]:
from optuna.visualization import plot_slice
plot_slice(optuna_study, target=lambda t: t.values[1])

In [ ]:
from optuna.visualization import plot_edf
plot_edf(optuna_study, target=lambda t: t.values[1])

In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(optuna_study, target=lambda t: t.values[1])

In [ ]:
from optuna.visualization import plot_param_importances
plot_param_importances(optuna_study, target=lambda t: t.values[0])